In [1]:
# import datetime as dt
# import pandas_datareader as pdr

# ## List of symbols to grab
# today = dt.date.today().strftime("%Y-%m-%d")
# today

# Using JupyterDash to Develop App

- 12/29/20

In [ ]:

import numpy as np
np.random.seed(321)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12,4)

import plotly.express as px
import plotly.io as pio
pio.templates

In [ ]:
pio.templates.default = "seaborn"

In [ ]:
# %conda install -c anaconda pandas-datareader
import pandas_datareader as pdr

stock = 'GOOGL'
example = pdr.DataReader(stock,'yahoo','2012','2020').reset_index()
example

In [ ]:
pfig = px.scatter(example,x='Date',y='Adj Close',title=stock)
pfig

# Basic App

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

In [ ]:
# !pip install -U jupyter_plotly_dash
from jupyter_plotly_dash import JupyterDash

In [ ]:
app = JupyterDash('Example',serve_locally=False)
app.layout = html.Div(children = [
    html.H1('Our AMAZING Dashboard'),
    html.H2("Lets see if we can make something interesting..."),
    dcc.Graph(figure=pfig)
])
# app

In [ ]:
# help(html.Div)

# Extending to Multiple Stocks

In [ ]:
## List of symbols to grab
symbols = ['FB','AAPL','GOOGL','AMZN','MSFT']
data = {}
for stock in symbols:
    data[stock] = pdr.DataReader(stock,'yahoo','2012','2021')['Adj Close']
df = pd.DataFrame(data).reset_index()
df

In [ ]:
def plot_stocks(y=['MSFT','AAPL']):
    pfig = px.scatter(df,x='Date',y=y)
    return pfig
    
plot_stocks()

## App with Dropdown

In [ ]:
def plot_stocks(stocks=['FB','AAPL']):
    pfig = px.scatter(df,x='Date',y=stocks)
    return pfig

In [ ]:
def make_options(menu_choices,ignore_cols=['Date']):
    """Returns list of dictionary with {'label':menu_choice,'value':menu_choice}"""
    options = []
    for choice in menu_choices:
        if choice not in ignore_cols:
            options.append({'label':choice,'value':choice})
    return options

make_options(df)

In [ ]:
app = JupyterDash('Example', 
                  serve_locally=False)
defaults = ['MSFT']
app.layout = html.Div(children = [
    html.H1('Our AMAZING Dashboard'),
    html.H2("Lets see if we can make something interesting..."),
    
    html.Div(children=[
        
        html.Div(id='menu',children=[
            
        dcc.Dropdown(id='choose_stocks',placeholder=defaults,multi=True,
                     options=make_options(df.columns))
        ]),
    
        dcc.Graph(id='line_plot',figure=plot_stocks(stocks=defaults))

    ])
])


@app.callback(Output('line_plot','figure'),
              [Input('choose_stocks','value')])
def update_stocks(stocks):
    return plot_stocks(stocks=stocks)
# app

### Functionizing Loading in of Data

In [ ]:
# def make_data_plo

In [ ]:
# display(plot_df)
# plot_stocks_df(plot_df)

### Adding Date Selection
- https://dash.plotly.com/dash-core-components/datepickerrange

In [1]:
import datetime as dt
from datetime import date
import pandas_datareader as pdr

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State


import numpy as np
np.random.seed(321)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12,4)

import plotly.express as px
import plotly.io as pio
pio.templates


today = dt.date.today().strftime("%Y-%m-%d")
def get_data(start_date='2012-02-01',end_date=today,symbols = ['FB','AAPL','GOOGL','AMZN','MSFT']):
    
    data = {}
    for stock in symbols:
        try:
            data[stock] = pdr.DataReader(stock,'yahoo',start_date,end_date)['Adj Close']
        except:
            print('Error with stock: '+stock)
    df = pd.DataFrame(data).reset_index()
    return df


def plot_stocks_df(df=None,stocks=['FB','AAPL']):
    if df is None:
        df = get_data(stocks)#.reset_index()
    stocks_exist = [s for s in stocks if s in df.columns]
    pfig = px.scatter(df,x='Date',y=stocks_exist)
    return pfig

from jupyter_plotly_dash import JupyterDash
app = JupyterDash('Example', 
                  serve_locally=False)
defaults = ['MSFT','AMZN']
# df = get_data(date(2012, 1, 1))

app.layout = html.Div(children = [
    html.H1('Our AMAZING Dashboard'),
    
    html.Div(children=[
        html.Div(style={"border":'1px solid black','padding':'3px'},id='menu',children=[
            html.H3('Enter stock symbols, separated by a comma.\nPress Enter to update.'),

            dcc.Input(id='choose_stocks',value='MSFT,AMZN',
                      placeholder='MSFT,AAPL',#,
                         style={'width': '90%', 'height': 50}),           

            dcc.DatePickerRange( id='my-date-picker-range',
                                start_date=date(2012,1,1),
                                min_date_allowed=date(2010, 1, 1),
                                max_date_allowed=today,#date.today(),#date(2021),
                                end_date=today,style={'width':'50%'}),#date.today()), #date(2017, 8, 25)),
            html.Button('Submit',id='submit',style={'size':'40 px'})
    
        ]),
        
        dcc.Graph(id='line_plot')

    ])
])

@app.callback(Output('line_plot','figure'),
              [Input('submit','n_clicks')],
              [State('choose_stocks','value'),
               State('my-date-picker-range', 'start_date'),
               State('my-date-picker-range', 'end_date')])
def update_stocks(n_clicks, stocks,start_date,end_date):
    stocks = stocks.split(',')
    plot_df = get_data(start_date,end_date,symbols=stocks)
    return plot_stocks_df(df=plot_df,stocks=stocks)#,start_date,end_date)
    
app

In [ ]:
# stocks = ['AMZN','MSFT','garbage']
# plot_df = get_data('2010','2013',stocks)

# stocks_exist = [s for s in stocks if s in plot_df.columns]
# plot_stocks_df(plot_df,stocks)